WHAT IS DARLI

WHAT TO EXPECT NEXT

The model in DARLi is a building block that can be further extended with external forces and contacts, different selector matrices, friction in joints, state space and parametric representations, please follow up to the next steps of tutorial to get in to that.

- General-info
- Backends
- Models 
- Floating Base,Conventions 
- Further Read and Structure

- Model of Poly-Articulated system
- Conventions
- rnea and aba
- Floating Base



OTHER TUTORS:
* MODELS, BODIES, SELECTORS ETC
* FUNCTIONAL, CODE GEN, BATCHED
* STATE SPACE 
* PARAMETRIC REPRESENTATIONS


## **Introduction to DARLi, Basics of Backends and Models** 


The DARLi (Differentiable Articulated Robotics Library) is a wrapper around the CasADi kinematics/dynamics and Pinocchio libraries. Its primary goal is to facilitate the creation of differentiable models for robotic systems, provided a URDF (Unified Robot Description Format) file. **DARLi is not an implementation of mechanics-oriented** algorithms, such as Featherstone's Articulated Body or the Recursive Newton-Euler. Instead, we rely on the efficient implementations provided by [Pinocchio](https://github.com/stack-of-tasks/pinocchio/tree/master) and offer a wrapper that grants easy access to a suite of features that we and our colleagues have found useful in practice of trajectory planning, feedback control, and system identification over poly-articulated robots.

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

**Note**: You have to restart kernel after `robot_descriptions` is installed

In [1]:

%%capture
!pip3 install robot_descriptions
from robot_descriptions import panda_description, panda_mj_description

### Backends 
The typical DARLi model is based on instance of Backend. Backend take `urdf` file as input and defines the way of computing two main functions: Articulated Bpdy Algorithm `aba` and Recursive Newton Euler `rnea`, as well as severals utility functions. These are soledly based on Pinnochio. For now there are two backends are available:

TODO: WHAT IS RNEA AND ABA

- `PinocchioBackend` - Provide interface to the Pinnochio python bindings. And may be useufull if one wants to obtain fast **numericall** computations. 
- `CasadiBackend` - The CasADi wrapper around Pinnochiio, this is used to cimpute dynamics in **symbolical** fashion, this allows for seeamless calculation of derivative and potentially used in trajectory optimization and optimal control.


The acces to backends is as simple as:

In [2]:
from darli.backend import CasadiBackend, PinocchioBackend

# Building the Pinnochio backend
pin_backend = PinocchioBackend(panda_description.URDF_PATH)
# Building the CasADi backend
cas_backend = CasadiBackend(panda_mj_description.MJCF_PATH)

In [3]:
cas_backend.nq

9

In [4]:
pin_backend.nq
pin_backend.nv
pin_backend.aba
pin_backend.rnea

<bound method PinocchioBackend.rnea of <darli.backend._pinocchio.PinocchioBackend object at 0x7992e63a07f0>>

### Floating Base, Fixed joints and Conventions: 



TODO: Something on floating base and conventions

In [5]:
from robot_descriptions import go2_description

In [6]:
go2_cas = CasadiBackend(go2_description.URDF_PATH)

In [7]:
print(f"Dimensions of model, nq: {go2_cas.nq} nv: {go2_cas.nv}")

Dimensions of model, nq: 12 nv: 12


In [8]:
from darli.backend import JointType

go2_cas_floating = CasadiBackend(
    go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER
)

print(
    f"Dimensions of floating model, nq: {go2_cas_floating.nq} nv: {go2_cas_floating.nv}"
)

Dimensions of floating model, nq: 19 nv: 18


In [9]:
cas_backend = CasadiBackend(go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER)

The model in DARLi is a building block that can be further extended with external forces and contacts, different selector matrices, friction in joints, state space and parametric representations, please follow up to the next steps of tutorial to get in to that.

- General-info
- Backends
- Models 
- Floating Base,Conventions 
- Further Read and Structure


- Model of Poly-Articulated system
- Conventions
- rnea and aba
- Floating Base


### Conventions



### Backends

In [10]:
from darli.model import Model

In [12]:
casadi_model = Model(cas_backend)

In [13]:
casadi_model.com()

CoM(position=SX(@1=1, @2=2, @3=(@2*q_4), @4=(@3*q_4), @5=(@2*q_5), @6=(@5*q_5), @7=0.0036612, @8=cos(q_17), @9=-0.00430848, @10=0.00084392, @11=cos(q_18), @12=-0.02623, @13=sin(q_18), @14=(@9+((@10*@11)+(@12*@13))), @15=sin(q_17), @16=-0.0376704, @17=-0.041322, @18=(@16+(((@12*@11)-(@10*@13))+@17)), @19=-0.391442, @20=cos(q_14), @21=cos(q_15), @22=sin(q_15), @23=(@9+((@10*@21)+(@12*@22))), @24=sin(q_14), @25=(@16+(((@12*@21)-(@10*@22))+@17)), @26=-0.0036612, @27=cos(q_11), @28=cos(q_12), @29=sin(q_12), @30=(@9+((@10*@28)+(@12*@29))), @31=sin(q_11), @32=(@16+(((@12*@28)-(@10*@29))+@17)), @33=0.391442, @34=cos(q_8), @35=cos(q_9), @36=sin(q_9), @37=(@9+((@10*@35)+(@12*@36))), @38=sin(q_8), @39=(@16+(((@12*@35)-(@10*@36))+@17)), @40=((((0.146694+((@7+((@8*@14)+(@15*@18)))+@19))+((@7+((@20*@23)+(@24*@25)))+@19))+((@26+((@27*@30)+(@31*@32)))+@33))+((@26+((@34*@37)+(@38*@39)))+@33)), @41=(@3*q_3), @42=(@5*q_6), @43=-0.104019, @44=cos(q_16), @45=sin(q_16), @46=-7.119e-05, @47=(@46+((@8*@18)-(@

One can retrive basic model info as follows:

In [ ]:
nq = casadi_model.nq  # dimensionality of configuration
nv = casadi_model.nv  # dimensionality of generilized velocities
nu = casadi_model.nu  # dimensionality  of control inputs
q_min, q_max = (
    casadi_model.q_min,
    casadi_model.q_max,
)  # minimal and maximal limits on joint pos
nq, nv, nu

In [ ]:
joint_names = casadi_model.joint_names  # names of the joints
joint_names

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [ ]:
inertia = casadi_model.inertia
gravity_vector = casadi_model.gravity
bias_force = casadi_model.bias_force
coriolis_matrix = casadi_model.coriolis_matrix
coriolis = casadi_model.coriolis

Each of the above define the CasAdi functions:

And can be evaluated both numerically and Functionalally:

In [ ]:
inertia = casadi_model.inertia
import numpy as np
import casadi as cs

# Functionalal computation
print("Functional:", inertia(cs.SX.sym("q", nq)))
# Numerical computation
# print("Numerical:", inertia(np.random.randn(nq)))

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [ ]:
casadi_model.energy()

In [ ]:
com_position = casadi_model.com().position
com_velocity = casadi_model.com().velocity
com_jacobian = casadi_model.com().jacobian
com_jacobian_dt = casadi_model.com().jacobian_dt
potential_energy = casadi_model.energy().potential
kinetic_energy = casadi_model.energy().kinetic
lagrangian = casadi_model.lagrangian()

# TODO
# momentum

In [ ]:
com_jacobian_dt

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [ ]:
casadi_model.contact_qforce

In [ ]:
casadi_model.inverse_dynamics()

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [ ]:
casadi_model.contact_qforce

In [ ]:
casadi_model.forward_dynamics()

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [ ]:
import numpy as np

print(f"Old input dimensions: {casadi_model.nu}")
S = np.random.randn(casadi_model.nv, casadi_model.nv + 2)
casadi_model.update_selector(S)
print(f"New input dimensions: {casadi_model.nu}")

In [ ]:
casadi_model.qfrc_u

In [ ]:
casadi_model.forward_dynamics

In [ ]:
casadi_model.update_selector(passive_joints=range(3))
print(f"New input dimensions: {casadi_model.nu}")
print(f"New selector:\n {casadi_model.selector}")

In [ ]:
casadi_model.forward_dynamics

#### **Bodies and Contacts**

In [ ]:
casadi_model.add_body(["link06"])
casadi_model.bodies

In [ ]:
model.body("link06")

### Frames

In [ ]:
model.body("link06").linear_acceleration

In [ ]:
model.body("link06").linear_acceleration.local

In [ ]:
model.body("link06").jacobian.local

In [ ]:
model.body("link03").jacobian.local
model.body("link03").jacobian_dt.local
model.body("link03").linear_velocity.local
model.body("link03").angular_velocity.local
model.body("link03").linear_acceleration.local
model.body("link03").angular_acceleration.local

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

In [ ]:
model.state_space.state_jacobian

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

In [ ]:
model.body("link06").contact.cone.linear()

In [ ]:
model.add_body({"ee": "link06"})
model.bodies

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.